<a href="https://colab.research.google.com/github/jovanadobreva/Labs-I2DS/blob/main/Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Decision Trees and Gradient Boosting

## Setting up the Environment

For this laboratory exercise, you will need to install the Anaconda package & environment manager. We will install a minimal distribution, [Miniconda](https://docs.conda.io/projects/miniconda/en/latest/). Choose the adequate distribution for your operating system, download and install it.

Or use the following commands:

### Windows
```shell
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-Windows-x86_64.exe -o miniconda.exe
start /wait "" miniconda.exe /S
del miniconda.exe
```

### Linux
```shell
mkdir -p ~/miniconda3
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

### macOS

```shell
mkdir -p ~/miniconda3
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-arm64.sh -o ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm -rf ~/miniconda3/miniconda.sh
```

For both Linux and macOS after installing, initialize your newly-installed Miniconda. The following commands initialize for bash and zsh shells:

```shell
~/miniconda3/bin/conda init bash
~/miniconda3/bin/conda init zsh
```


Once you have installed miniconda, run the following commands to create an environment:
```bash
conda create --name myenv
```

'myenv' is the name of the environment, you can change the name however you want.

When conda asks you to proceed, type y

After successfully creating the environment, activate it with the following command:
```bash
conda activate myenv
```

For more detailed information you can read the [documentation](https://conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#creating-an-environment-with-commands).

Now, once the environment is activated, proceed to install the required libraries.

```bash
pip install numpy pandas scikit-learn xgboost matplotlib seaborn gdown
```

In the next step, we need to add the environment to jupyter. Use the following commands to install ipykernel and add the environment to ipykernel.

```bash
pip install ipykernel
```
```bash
python -m ipykernel install --name=myenv
```


Next, start Jupyter Notebook, download this starter notebook and open it. On the dropdown menu in the Kernel tab choose the name of the environment you created, like in the picture below.


![jupyter](https://drive.google.com/uc?export=view&id=1N-27jjlIgpTILi-_6lny7ng8sE52SAZx)


## Download and Read the Dataset

run the code below for downloading the dataset

In [ ]:
!gdown 1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx

Downloading...
From: https://drive.google.com/uc?id=1boIax8d9Sat6OJzkiIjjpqmtSZKuRYrx
To: C:\Users\mgadz\ElectricCarData.csv

  0%|          | 0.00/8.20k [00:00<?, ?B/s]
100%|##########| 8.20k/8.20k [00:00<?, ?B/s]


### Import the required libraries

In [ ]:
import pandas as pd
import numpy as np

### Read the dataset

CONTEXT:
This is a dataset of electric vehicles.

It contains the following columns:


*   Brand
*   Model
*   AccelSec - Acceleration as 0-100 km/h
*   TopSpeed_KmH - The top speed in km/h
*   Range_Km - Range in km
*   Efficiency_WhKm - Efficiency Wh/km
*   FastCharge_KmH - Charge km/h
*   RapidCharge - Yes / No
*   PowerTrain - Front, rear, or all wheel drive
*   PlugType
*   BodyStyle - Basic size or style
*   Segment - Market segment
*   Seats - Number of seats
*   PriceEuro - Price in Germany before tax incentives




TASK:
Predict the target 'PriceEuro' and compare the performance of the DecisionTreeRegressor and the XGBRegressor models.

In [ ]:
data = pd.read_csv('/Users/mgadz/ElectricCarData.csv')

In [ ]:
data.head()

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Segment,Seats,PriceEuro
0,Tesla,Model 3 Long Range Dual Motor,4.6,233,450,161,940,Yes,AWD,Type 2 CCS,Sedan,D,5,55480
1,Volkswagen,ID.3 Pure,10.0,160,270,167,250,Yes,RWD,Type 2 CCS,Hatchback,C,5,30000
2,Polestar,2,4.7,210,400,181,620,Yes,AWD,Type 2 CCS,Liftback,D,5,56440
3,BMW,iX3,6.8,180,360,206,560,Yes,RWD,Type 2 CCS,SUV,D,5,68040
4,Honda,e,9.5,145,170,168,190,Yes,RWD,Type 2 CCS,Hatchback,B,4,32997


### Encode string variables

In [ ]:
categorical_columns = ['Brand', 'Model', 'RapidCharge', 'PowerTrain', 'PlugType', 'BodyStyle', 'Segment']

data_encoded = pd.get_dummies(data, columns=categorical_columns)

print(data_encoded.head())


   AccelSec  TopSpeed_KmH  Range_Km  Efficiency_WhKm FastCharge_KmH  Seats  \
0       4.6           233       450              161            940      5   
1      10.0           160       270              167            250      5   
2       4.7           210       400              181            620      5   
3       6.8           180       360              206            560      5   
4       9.5           145       170              168            190      4   

   PriceEuro  Brand_Aiways   Brand_Audi   Brand_BMW   ...  BodyStyle_Sedan  \
0      55480          False        False       False  ...             True   
1      30000          False        False       False  ...            False   
2      56440          False        False       False  ...            False   
3      68040          False        False        True  ...            False   
4      32997          False        False       False  ...            False   

   BodyStyle_Station  Segment_A  Segment_B  Segment_C  Segment

## Split the dataset for training and testing in ratio 80:20

In [ ]:
features = ['AccelSec', 'TopSpeed_KmH', 'Range_Km', 'Efficiency_WhKm', 'FastCharge_KmH', 'Seats']
from sklearn.model_selection import train_test_split

X = data_encoded[features]
y = data_encoded['PriceEuro']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.replace('-', np.nan, inplace=True)
X_train = X_train.astype(float)



## Initialize the DecisionTreeRegressor model, and use the fit function for training the model.

Add values for the parameters max_depth, min_samples_split, and max_features.

Fit the model using the fit function


In [ ]:
from sklearn.tree import DecisionTreeRegressor

regressor = DecisionTreeRegressor(max_depth=5, min_samples_split=2, max_features=1)

regressor.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=5, max_features=1)

## Predict the outcomes for X test

In [ ]:
y_pred = regressor.predict(X_test)

print(y_pred)

[ 33558.42105263  53364.2         58397.          85000.
  53364.2         58397.          33558.42105263  58397.
  58397.          83080.          33558.42105263  58397.
  33558.42105263  33558.42105263  58397.          33558.42105263
 163770.75        85000.          58397.          58397.
  33558.42105263]


## Assess the model performance, by using sklearn metrics for regression

In [ ]:
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')


Mean Absolute Error: 17316.004135338346


## Initialize the XGBRegressor model, and use the fit function

Add values for the parameters: n_estimators, max_depth, learning_rate, and set the objective to "reg:squarederror"

Fit the model using the fit function

In [ ]:
import xgboost as xgb

data_encoded['FastCharge_KmH'] = pd.to_numeric(data_encoded['FastCharge_KmH'], errors='coerce')

features = ['AccelSec', 'TopSpeed_KmH', 'Range_Km', 'Efficiency_WhKm', 'FastCharge_KmH', 'Seats']

X_train, X_test, y_train, y_test = train_test_split(data_encoded[features], data_encoded['PriceEuro'], test_size=0.2)

xgb_regressor = xgb.XGBRegressor(
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    objective='reg:squarederror',
)

# Fit the model
xgb_regressor.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

## Predict the outcomes for X test

In [ ]:
y_pred_xgb = xgb_regressor.predict(X_test)

print(y_pred_xgb)

[ 65611.96   57028.285  36563.68   20908.51   47102.902  52489.293
  38538.996  70505.02   35633.383  39410.066  22451.717  33115.03
  29544.293 101956.516  64151.605  63508.523  36563.68  148193.8
  70917.8    31226.533  30024.43 ]


## Assess the model performance, by using sklearn metrics for regression

In [ ]:
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)

print(f'Mean Absolute Error (XGBRegressor): {mae_xgb}')

Mean Absolute Error (XGBRegressor): 13808.453218005952


## Compare the performances of both model for at least three regression metircs

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae_tree = mean_absolute_error(y_test, y_pred)
mse_tree = mean_squared_error(y_test, y_pred)
r2_tree = r2_score(y_test, y_pred)

print(f'DecisionTreeRegressor Metrics:')
print(f'Mean Absolute Error: {mae_tree}')
print(f'Mean Squared Error: {mse_tree}')
print(f'R-squared: {r2_tree}')

# XGBRegressor
y_pred_xgb = xgb_regressor.predict(X_test)

mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print('\nXGBRegressor Metrics:')
print(f'Mean Absolute Error: {mae_xgb}')
print(f'Mean Squared Error: {mse_xgb}')
print(f'R-squared: {r2_xgb}')


DecisionTreeRegressor Metrics:
Mean Absolute Error: 37532.97556390978
Mean Squared Error: 3066707258.2696404
R-squared: -0.48645445156494005

XGBRegressor Metrics:
Mean Absolute Error: 13808.453218005952
Mean Squared Error: 947123633.755849
R-squared: 0.5409225521013333
